# Import libraries

In [ ]:
# Upload File using ipyfilechooser library
from ipyfilechooser import FileChooser
# Get the root directory of the project
from pyprojroot import here
from IPython.display import display, Javascript
# Copy File
import shutil
# Widget Packages
import ipywidgets as widgets
# In case widget extension not working
# jupyter nbextension enable --py widgetsnbextension
import os

# Data Exploration Section
R2 (Epic): As a user, I want a "Data Exploration" section in the notebook so that I can load and display video data from the TSU project.

## Video Upload / Choose using ipyfilechooser
R2 (Story): As a user, I want to upload/choose files from the data folder through an appropriate UI component (E.g. Browse files) in a notebook code cell so that I can pick and choose the video data I would like to process

In [ ]:
# from ipyfilechooser import FileChooser
def videoselectorinput():
    starting_directory = './data'
    chooser = FileChooser(starting_directory)
    display(chooser)
    return chooser
    
def videoselectoroutput():
    starting_directory = './pipeline/video/output'
    chooser = FileChooser(starting_directory)
    display(chooser)
    return chooser

In [ ]:
video = videoselectorinput()

## Upload selected file to the data folder (If needed)

In [ ]:
# Upload Function
# from pyprojroot import here
# import shutil
def upload(video):
    print(video.selected)
    source = video.selected
    # Source path
    # Destination path
    destination = (here("./data"))

    # Copy file from the selected path
    try:
        shutil.copy(source, destination)
        print("File copied successfully.")

    # If source and destination are same
    except shutil.SameFileError:
        print("Source and destination represents the same file.")

    # If destination is a directory.
    except IsADirectoryError:
        print("Destination is a directory.")

    # If there is any permission issue
    except PermissionError:
        print("Permission denied.")

    # For other errors
    except:
        print("Error occurred while copying file.")

upload(video)

## Video Playback
R2 (Story): As a user, I want to see video playback of the chosen video file in an output cell so that I can check if it is the right video data I would like to process

In [ ]:
# Select Video
video = videoselectorinput()

In [ ]:
print(video.selected)
print(video.selected_filename)
print(video.selected_path)

In [ ]:
from IPython.display import Video

full_path = video.selected
Video(full_path, embed=True, width=540, html_attributes="controls muted autoplay")

# Inference Section

R3 (Epic): As a user, I want to have an "Inference" section in the notebook so that I can perform inference using a pretrained HOI ML model based on the TSU project.

## Load a pretrain model
R3 (Story): As a user, I want to load a pre-trained model using an appropriate UI component so that I can easily load the model.

In [ ]:
"""
Select the model
"""
modelList = [] 

# Select from the list of model in the pipeline/models folder
for x in os.listdir("./pipeline/models"): 
    modelList += [x]

# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)
modelDropdown = widgets.Dropdown(
    options=modelList,
    value=modelList[0],
    description='Model:')
# Function on what happen when confirm is been click.
def selectWidgetSet(b):
    print("Selected: " , modelDropdown.value)

confirmButton.on_click(selectWidgetSet)
modelBox = widgets.VBox([widgets.HBox([modelDropdown, confirmButton])])
modelBox

In [ ]:
print(modelDropdown.value)

## Choose Input video to load into TSU Project
R3 (Story): As a user, I want to choose an  input video files and other related input files, using an appropriate UI component, from the TSU project so that the system is able to pass the right files to the model.

In [ ]:
# Select Video
video = videoselectorinput()

In [ ]:
print(video.selected)
print(video.selected_filename)
print(video.selected_path)

## Run the model 
R3 (Story): As a user, I want to see inference results in the form of an output video with captions that indicate the current detected activity in each video frame so that I am able to see the inference results clearly on the screen.

In [ ]:
# Testing
!echo $modelDropdown.value
!echo $video.selected_filename
!echo $video.selected
# Get the directory of the loadmodel
loadmodel = './models/' + modelDropdown.value
!echo $loadmodel

In [ ]:
!cd ./pipeline && python test.py -input_video_full_path $video.selected -input_video_file $video.selected_filename -model $modelDropdown.value -load_model $loadmodel

## Output Video to view the inference result 

In [ ]:
# Select output video
video = videoselectoroutput()

In [ ]:
print(video.selected)
print(video.selected_filename)
print(video.selected_path)

In [ ]:
from IPython.display import Video

full_path = video.selected
Video(full_path, embed=True, width=540, html_attributes="controls muted autoplay")

# Feature Extraction Section


## Running main feature-extraction function
After which, we will be running the main feature-extraction function to extract features from valid videos listed in valid_videos.txt to create RGB/FLOW/RGB+FLOW .npy files for training later on.

In [ ]:
!cd ./i3d-feature-extraction && python main.py feature_type=r21d device="cuda:0" on_extraction=save_numpy streams=rgb output_path=./output/RGB file_with_video_paths=./sample/sample_video_paths.txt

display(Javascript("""
require(
    ["base/js/dialog"], 
    function(dialog) {
        dialog.modal({
            title: 'Feature Extraction',
            body: 'Feature Extraction Successful! RGB npy files saved to i3d-feature-extraction/output/RGB',
            buttons: {
                'Okay': {}
            }
        });
    })
"""))

## Running validate_train_test.py
Running validate_train_test.py to remove video IDs from smarthome_cs_51.json file and create an updated version called smarthome_cs_51_v2.json

In [ ]:
!cd ./i3d-feature-extraction && python validate_train_test.py

# Training Section
R4 (Epic): As a user, I want to create a "Training" section in the netbook so that I can train a HOI ML model based on the TSU project.

# Testing Section
R5 (Epic): As a user, I want to have a "Testing" section in the notebook so that I can evaluate a trained model based on the TSU project.

# NVIDIA STEP Section
R6 (Epic): As a user, I want to able to configure the notebook using appropriate UI elements coupled with the right .py modules so that R2-5 can be performed based on another pipeline, e.g., the NVIDIA STEP pipeline 

## Pipeline Selection
R6(Story): As a user, I want to create appropriate UI elements to allow for switching pipelines so that I can test the different models



In [ ]:
w = widgets.Dropdown(
    options=['TSU', 'NVIDIA SMarthome'],
    value='TSU',
    description='Pipeline:',
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])

w.observe(on_change)

display(w)

## Nvidida Step Pipeline
R6(Story): As a user, I want to ensure selected pipeline's dependencies are changed to ensure the right dependencies are given to the appropriate models so that the selected model will be run

In [ ]:
%cd ./NVIDIA-STEP-MODEL/STEP

In [ ]:
!pip install ninja

In [ ]:
!pip install cuda-python

In [ ]:
!ipython setup.py build develop

In [ ]:
!python demo.py

In [ ]:
from IPython.display import Image
Image(filename='datasets/demo/frames/results/2/frame0000.jpg') 

T05-119 As a user, I want to have multiple dropdowns to select the input (1) dataset (2) video so that I can select the dataset videos easily

In [ ]:

trim_untrim = [] 
directory = "./data"

# Select from the list of model in the pipeline/models folder
for x in os.listdir(directory): 
    trim_untrim += [x]

# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)

trimUntrimDropdown = widgets.Dropdown(
    options=trim_untrim,
    description='Trim or Untrim:')
# Function on what happen when confirm is been click.
def selectTrimSet(b):
    #print("B: ", b)
    global trimValue
    trimValue = trimUntrimDropdown.value
    print("Selected: " , trimUntrimDropdown.value)

confirmButton.on_click(selectTrimSet)
trimBox = widgets.VBox([widgets.HBox([trimUntrimDropdown, confirmButton])])
trimBox




In [ ]:

directory = './data' + '/'+ trimValue
print("Current File Path: ", directory)
data_set = os.listdir(directory)

# Select from the list of model in the pipeline/models folder
#directory = directory + '/'+ trimUntrimDropdown.value


# Widgets
confirm1Button = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)

datasetDropdown = widgets.Dropdown(
    options=data_set,
    description='Dataset:')
# Function on what happen when confirm is been click.
def selectDataSet(b):
    global datasetValue
    datasetValue = datasetDropdown.value
    print("Selected: " , datasetDropdown.value)

confirm1Button.on_click(selectDataSet)
datasetBox = widgets.VBox([widgets.HBox([datasetDropdown, confirm1Button])])
datasetBox


In [ ]:
print(datasetValue)
directory = './data' + '/'+ trimValue + '/'+ datasetValue
print("Current File Path: ", directory)
data_set = os.listdir(directory)

# Select from the list of model in the pipeline/models folder
#directory = directory + '/'+ trimUntrimDropdown.value


# Widgets
confirm2Button = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)

inputDropdown = widgets.Dropdown(
    options=data_set,
    value=data_set[0],
    description='Dataset:')
# Function on what happen when confirm is been click.
def selectInputSet(b):
    global inputValue
    inputValue = inputDropdown.value
    print("Selected: " , inputDropdown.value)

confirm2Button.on_click(selectInputSet)
inputBox = widgets.VBox([widgets.HBox([inputDropdown, confirm2Button])])
inputBox